In [30]:
import pandas as pd
import numpy as np
from config import conn_host, conn_database, conn_user, conn_password
import mysql.connector
from IPython.display import clear_output
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [137]:
LEAGUE_ID = 39

START_SEASON = 2014
END_SEASON = 2023

In [138]:
def connect_to_db():
    return mysql.connector.connect(host=conn_host, 
                                     database=conn_database,
                                     user=conn_user,
                                     password=conn_password)

def execute_query(query, read_only = True):
    resp = None
    try:
        db = connect_to_db()
        if read_only:
            resp = pd.read_sql_query(query, db)
        else:
            mycursor = db.cursor()
            mycursor.execute(query)

            db.commit()
    except Exception as e:
        print(e)
    db.close()
    return resp

def execute_multiple_queries(queries):
    try:
        db = connect_to_db()
        mycursor = db.cursor()
        for query in queries:
            mycursor.execute(query)

        db.commit()
        db.close()
    except Exception as e:
        print(e)

In [139]:
execute_query("ALTER TABLE matches " +
              "ADD home_odds FLOAT NULL," +
              "ADD away_odds FLOAT NULL," +
              "ADD draw_odds FLOAT NULL;")
execute_query("ALTER TABLE matches " +
              "ADD over_odds FLOAT NULL," +
              "ADD under_odds FLOAT NULL;")
execute_query("ALTER TABLE matches " +
              "ADD handicap_size FLOAT NULL," +
              "ADD handicap_odds FLOAT NULL;")

Execution failed on sql 'ALTER TABLE matches ADD home_odds FLOAT NULL,ADD away_odds FLOAT NULL,ADD draw_odds FLOAT NULL;': 1060 (42S21): Duplicate column name 'home_odds'
Execution failed on sql 'ALTER TABLE matches ADD over_odds FLOAT NULL,ADD under_odds FLOAT NULL;': 1060 (42S21): Duplicate column name 'over_odds'
Execution failed on sql 'ALTER TABLE matches ADD handicap_size FLOAT NULL,ADD handicap_odds FLOAT NULL;': 1060 (42S21): Duplicate column name 'handicap_size'


In [140]:
teams = execute_query("SELECT * FROM teams")
display(teams)

,id,name
0,33,Man United
1,34,Newcastle
2,35,Bournemouth
3,36,Fulham
4,37,Huddersfield
...,...,...
415,12277,Ipatinga
416,13173,Chivas USA
417,13974,Santa Cruz
418,16489,Austin FC


In [141]:
lines_df = None

for season in range(START_SEASON, END_SEASON):
    if season == START_SEASON:
        lines_df = pd.read_csv(f"./lines/{LEAGUE_ID}/{season}.csv")
    else:
        lines_df = pd.concat([lines_df, pd.read_csv(f"./lines/{LEAGUE_ID}/{season}.csv")])
        
display(lines_df)

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,16/08/14,Arsenal,Crystal Palace,2.0,1.0,H,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,16/08/14,Leicester,Everton,2.0,2.0,D,1.0,2.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,16/08/14,Man United,Swansea,1.0,2.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,16/08/14,QPR,Hull,0.0,1.0,A,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,16/08/14,Stoke,Aston Villa,0.0,1.0,A,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,E0,25/02/2023,Bournemouth,Man City,1.0,4.0,A,0.0,3.0,A,...,2.35,1.50,1.99,1.94,2.07,1.86,2.12,1.95,2.01,1.88
237,E0,25/02/2023,Crystal Palace,Liverpool,0.0,0.0,D,0.0,0.0,D,...,2.06,0.75,2.04,1.89,2.02,1.90,2.13,1.95,2.00,1.88
238,E0,26/02/2023,Tottenham,Chelsea,2.0,0.0,H,0.0,0.0,D,...,1.80,0.00,2.16,1.78,2.16,1.80,2.20,1.86,2.10,1.80
239,E0,01/03/2023,Arsenal,Everton,4.0,0.0,H,2.0,0.0,H,...,2.00,-1.50,2.01,1.92,1.98,1.94,2.01,1.98,1.96,1.92


In [142]:
unique_values = lines_df.HomeTeam.unique().astype(str)
print(f"There are {lines_df.HomeTeam.nunique()} different teams. Check your database to modify it's names in order to correctly update sportsbook info. The unique teams are:")
print(np.sort(unique_values[unique_values != 'nan']))

There are 33 different teams. Check your database to modify it's names in order to correctly update sportsbook info. The unique teams are:
['Arsenal' 'Aston Villa' 'Bournemouth' 'Brentford' 'Brighton' 'Burnley'
 'Cardiff' 'Chelsea' 'Crystal Palace' 'Everton' 'Fulham' 'Huddersfield'
 'Hull' 'Leeds' 'Leicester' 'Liverpool' 'Man City' 'Man United'
 'Middlesbrough' 'Newcastle' 'Norwich' "Nott'm Forest" 'QPR'
 'Sheffield United' 'Southampton' 'Stoke' 'Sunderland' 'Swansea'
 'Tottenham' 'Watford' 'West Brom' 'West Ham' 'Wolves']


In [143]:
print('Columns:')
print(lines_df.columns.tolist())

Columns:
['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'BbOU', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5', 'BbAv<2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'PSCH', 'PSCD', 'PSCA', 'Time', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5',

In [144]:
def convert_to_date(date):
    date_format = '%d/%m/%Y' if len(date.split('/')[-1]) == 4 else '%d/%m/%y'
    return datetime.strptime(date, date_format).strftime('%d/%m/%Y')

selected_columns = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'PSH', 'PSD', 'PSA', 'P>2.5', 'P<2.5', 'AHh', 'PAHH', 'PAHA']
filtered_lines_df = lines_df.loc[:, selected_columns].rename({'FTHG': 'HOME_SCORE', 'FTAG': 'AWAY_SCORE', 'P<2.5': 'UNDER_2.5_ODDS', 'P>2.5': 'OVER_2.5_ODDS', 'PSH': 'HOME_ODDS', 'PSD': 'DRAW_ODDS', 'PSA': 'AWAY_ODDS', 'AHh': 'HANDICAP'}, axis=1)
filtered_lines_df['HANDICAP_ODDS'] = (filtered_lines_df['PAHH'] + filtered_lines_df['PAHA']) / 2
filtered_lines_df.drop(['PAHH', 'PAHA'], axis=1, inplace=True)
filtered_lines_df['HANDICAP'] = filtered_lines_df['HANDICAP'].abs()
filtered_lines_df = filtered_lines_df[filtered_lines_df['Date'].notna()]
filtered_lines_df['Date'] = pd.to_datetime(filtered_lines_df['Date'].apply(lambda x: convert_to_date(x)), format='%d/%m/%Y') 
display(filtered_lines_df)

,Date,HomeTeam,AwayTeam,HOME_SCORE,AWAY_SCORE,HOME_ODDS,DRAW_ODDS,AWAY_ODDS,OVER_2.5_ODDS,UNDER_2.5_ODDS,HANDICAP,HANDICAP_ODDS
0,2014-08-16,Arsenal,Crystal Palace,2.0,1.0,1.26,6.45,14.01,NaN,NaN,NaN,NaN
1,2014-08-16,Leicester,Everton,2.0,2.0,3.14,3.38,2.46,NaN,NaN,NaN,NaN
2,2014-08-16,Man United,Swansea,1.0,2.0,1.37,5.10,10.60,NaN,NaN,NaN,NaN
3,2014-08-16,QPR,Hull,0.0,1.0,2.48,3.26,3.22,NaN,NaN,NaN,NaN
4,2014-08-16,Stoke,Aston Villa,0.0,1.0,1.95,3.47,4.55,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
236,2023-02-25,Bournemouth,Man City,1.0,4.0,9.67,5.74,1.30,1.59,2.47,1.50,1.950
237,2023-02-25,Crystal Palace,Liverpool,0.0,0.0,4.86,3.99,1.76,1.83,2.07,0.75,1.960
238,2023-02-26,Tottenham,Chelsea,2.0,0.0,2.59,3.36,2.96,2.08,1.84,0.00,1.970
239,2023-03-01,Arsenal,Everton,4.0,0.0,1.38,5.01,9.10,1.86,2.04,1.25,1.960


In [145]:
def get_team_id(team):
    return teams[teams['name'].str.lower() == team.lower()].reset_index(drop=True).loc[0, 'id']

filtered_lines_df['HOME_ID'] = filtered_lines_df.HomeTeam.apply(lambda x: get_team_id(x))
filtered_lines_df['AWAY_ID'] = filtered_lines_df.AwayTeam.apply(lambda x: get_team_id(x))
display(filtered_lines_df)

,Date,HomeTeam,AwayTeam,HOME_SCORE,AWAY_SCORE,HOME_ODDS,DRAW_ODDS,AWAY_ODDS,OVER_2.5_ODDS,UNDER_2.5_ODDS,HANDICAP,HANDICAP_ODDS,HOME_ID,AWAY_ID
0,2014-08-16,Arsenal,Crystal Palace,2.0,1.0,1.26,6.45,14.01,NaN,NaN,NaN,NaN,42,52
1,2014-08-16,Leicester,Everton,2.0,2.0,3.14,3.38,2.46,NaN,NaN,NaN,NaN,46,45
2,2014-08-16,Man United,Swansea,1.0,2.0,1.37,5.10,10.60,NaN,NaN,NaN,NaN,33,76
3,2014-08-16,QPR,Hull,0.0,1.0,2.48,3.26,3.22,NaN,NaN,NaN,NaN,72,64
4,2014-08-16,Stoke,Aston Villa,0.0,1.0,1.95,3.47,4.55,NaN,NaN,NaN,NaN,75,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2023-02-25,Bournemouth,Man City,1.0,4.0,9.67,5.74,1.30,1.59,2.47,1.50,1.950,35,50
237,2023-02-25,Crystal Palace,Liverpool,0.0,0.0,4.86,3.99,1.76,1.83,2.07,0.75,1.960,52,40
238,2023-02-26,Tottenham,Chelsea,2.0,0.0,2.59,3.36,2.96,2.08,1.84,0.00,1.970,47,49
239,2023-03-01,Arsenal,Everton,4.0,0.0,1.38,5.01,9.10,1.86,2.04,1.25,1.960,42,45


In [146]:
game_errors = []
update_queries = []

for index, row in filtered_lines_df.iterrows():
    try:
        date_minus_one_day = row['Date'] - timedelta(days=1)
        date_plus_one_day = row['Date'] + timedelta(days=1)
        game_id = execute_query(f"SELECT id from matches WHERE (DATE_FORMAT(date,'%d/%m/%Y') = '{row['Date'].strftime('%d/%m/%Y')}' OR DATE_FORMAT(date,'%d/%m/%Y') = '{date_minus_one_day.strftime('%d/%m/%Y')}' OR DATE_FORMAT(date,'%d/%m/%Y') = '{date_plus_one_day.strftime('%d/%m/%Y')}') AND home_id = {row['HOME_ID']} AND away_id = {row['AWAY_ID']}").iloc[0]['id']
        if not np.isnan(row['HOME_ODDS']) and not np.isnan(row['AWAY_ODDS']) and not np.isnan(row['DRAW_ODDS']): update_queries.append(f"UPDATE matches SET home_odds = {row['HOME_ODDS']}, away_odds = {row['AWAY_ODDS']}, draw_odds = {row['DRAW_ODDS']} WHERE id = {game_id}")
        if not np.isnan(row['OVER_2.5_ODDS']) and not np.isnan(row['UNDER_2.5_ODDS']): update_queries.append(f"UPDATE matches SET over_odds = {row['OVER_2.5_ODDS']}, under_odds = {row['UNDER_2.5_ODDS']} WHERE id = {game_id}")
        if not np.isnan(row['HANDICAP_ODDS']) and not np.isnan(row['HANDICAP']): update_queries.append(f"UPDATE matches SET handicap_size = {row['HANDICAP']}, handicap_odds = {row['HANDICAP_ODDS']} WHERE id = {game_id}")
    except Exception as e:
        new_item = dict()
        new_item['error'] = e
        new_item['match'] = row
        game_errors.append(new_item)
    clear_output(wait=True)
    print(f"{index}/{len(filtered_lines_df)}")

240/3281


In [147]:
if len(game_errors) == 0:
    print('There were no errors while reading games from the Football Co Data csvs.')
else:
    print('The following games had an error while trying to read games from the Football Co Data csvs:')
    for item in game_errors:
        print(f"\n{item['match']['Date']}: {item['match']['HomeTeam']}({item['match']['HOME_ID']}) {item['match']['HOME_SCORE']} x {item['match']['AWAY_SCORE']} {item['match']['AWAY_ID']}({item['match']['AwayTeam']})")
        print(f"ERROR: {item['error']}")

The following games had an error while trying to read games from the Football Co Data csvs:

2023-01-12 00:00:00: Fulham(36) 2.0 x 1.0 49(Chelsea)
ERROR: single positional indexer is out-of-bounds

2023-01-18 00:00:00: Crystal Palace(52) 1.0 x 1.0 33(Man United)
ERROR: single positional indexer is out-of-bounds

2023-01-19 00:00:00: Man City(50) 4.0 x 2.0 47(Tottenham)
ERROR: single positional indexer is out-of-bounds

2023-02-08 00:00:00: Man United(33) 2.0 x 2.0 63(Leeds)
ERROR: single positional indexer is out-of-bounds

2023-02-15 00:00:00: Arsenal(42) 1.0 x 3.0 50(Man City)
ERROR: single positional indexer is out-of-bounds

2023-03-01 00:00:00: Arsenal(42) 4.0 x 0.0 45(Everton)
ERROR: single positional indexer is out-of-bounds

2023-03-01 00:00:00: Liverpool(40) 2.0 x 0.0 39(Wolves)
ERROR: single positional indexer is out-of-bounds


In [148]:
for q in update_queries:
    print(q)

UPDATE matches SET home_odds = 1.26, away_odds = 14.01, draw_odds = 6.45 WHERE id = 192683
UPDATE matches SET home_odds = 3.14, away_odds = 2.46, draw_odds = 3.38 WHERE id = 192678
UPDATE matches SET home_odds = 1.37, away_odds = 10.6, draw_odds = 5.1 WHERE id = 192677
UPDATE matches SET home_odds = 2.48, away_odds = 3.22, draw_odds = 3.26 WHERE id = 192679
UPDATE matches SET home_odds = 1.95, away_odds = 4.55, draw_odds = 3.47 WHERE id = 192680
UPDATE matches SET home_odds = 2.21, away_odds = 3.73, draw_odds = 3.34 WHERE id = 192681
UPDATE matches SET home_odds = 3.96, away_odds = 2.06, draw_odds = 3.55 WHERE id = 192682
UPDATE matches SET home_odds = 1.35, away_odds = 10.3, draw_odds = 5.48 WHERE id = 192684
UPDATE matches SET home_odds = 5.8, away_odds = 1.63, draw_odds = 4.28 WHERE id = 192685
UPDATE matches SET home_odds = 9.77, away_odds = 1.41, draw_odds = 4.81 WHERE id = 192686
UPDATE matches SET home_odds = 2.56, away_odds = 3.01, draw_odds = 3.36 WHERE id = 192687
UPDATE matc

UPDATE matches SET home_odds = 4.15, away_odds = 2.02, draw_odds = 3.51 WHERE id = 193013
UPDATE matches SET home_odds = 2.35, away_odds = 3.44, draw_odds = 3.29 WHERE id = 193016
UPDATE matches SET home_odds = 3.66, away_odds = 2.2, draw_odds = 3.39 WHERE id = 193015
UPDATE matches SET home_odds = 4.15, away_odds = 2.04, draw_odds = 3.44 WHERE id = 193005
UPDATE matches SET home_odds = 5.58, away_odds = 1.69, draw_odds = 3.98 WHERE id = 192946
UPDATE matches SET home_odds = 2.78, away_odds = 2.79, draw_odds = 3.32 WHERE id = 193018
UPDATE matches SET home_odds = 1.97, away_odds = 4.18, draw_odds = 3.64 WHERE id = 193017
UPDATE matches SET home_odds = 1.38, away_odds = 9.56, draw_odds = 5.2 WHERE id = 193019
UPDATE matches SET home_odds = 1.38, away_odds = 10.35, draw_odds = 5.23 WHERE id = 193023
UPDATE matches SET home_odds = 4.04, away_odds = 2.08, draw_odds = 3.44 WHERE id = 193020
UPDATE matches SET home_odds = 2.28, away_odds = 3.45, draw_odds = 3.43 WHERE id = 193021
UPDATE matc

UPDATE matches SET home_odds = 1.45, away_odds = 8.42, draw_odds = 4.71 WHERE id = 192541
UPDATE matches SET home_odds = 1.61, away_odds = 6.14, draw_odds = 4.26 WHERE id = 192537
UPDATE matches SET home_odds = 2.22, away_odds = 3.71, draw_odds = 3.33 WHERE id = 192543
UPDATE matches SET home_odds = 1.77, away_odds = 5.33, draw_odds = 3.74 WHERE id = 192544
UPDATE matches SET home_odds = 2.66, away_odds = 2.88, draw_odds = 3.36 WHERE id = 192539
UPDATE matches SET home_odds = 2.26, away_odds = 3.67, draw_odds = 3.28 WHERE id = 192538
UPDATE matches SET home_odds = 1.51, away_odds = 7.73, draw_odds = 4.39 WHERE id = 192542
UPDATE matches SET home_odds = 3.84, away_odds = 2.08, draw_odds = 3.58 WHERE id = 192545
UPDATE matches SET home_odds = 2.1, away_odds = 4.04, draw_odds = 3.37 WHERE id = 192546
UPDATE matches SET home_odds = 2.07, away_odds = 4.06, draw_odds = 3.44 WHERE id = 192549
UPDATE matches SET home_odds = 1.46, away_odds = 7.95, draw_odds = 4.79 WHERE id = 192553
UPDATE matc

UPDATE matches SET home_odds = 1.8, away_odds = 5.4, draw_odds = 3.57 WHERE id = 17557
UPDATE matches SET home_odds = 1.38, away_odds = 9.21, draw_odds = 5.29 WHERE id = 17552
UPDATE matches SET home_odds = 3.48, away_odds = 2.26, draw_odds = 3.45 WHERE id = 17553
UPDATE matches SET home_odds = 3.43, away_odds = 2.32, draw_odds = 3.37 WHERE id = 17554
UPDATE matches SET home_odds = 5.28, away_odds = 1.7, draw_odds = 4.1 WHERE id = 17551
UPDATE matches SET home_odds = 2.14, away_odds = 3.68, draw_odds = 3.55 WHERE id = 17555
UPDATE matches SET home_odds = 3.15, away_odds = 2.45, draw_odds = 3.39 WHERE id = 17556
UPDATE matches SET home_odds = 1.27, away_odds = 14.32, draw_odds = 6.25 WHERE id = 17550
UPDATE matches SET home_odds = 1.29, away_odds = 11.73, draw_odds = 6.29 WHERE id = 17547
UPDATE matches SET home_odds = 2.22, away_odds = 3.64, draw_odds = 3.37 WHERE id = 17548
UPDATE matches SET home_odds = 1.68, away_odds = 6.42, draw_odds = 3.68 WHERE id = 17549
UPDATE matches SET home

UPDATE matches SET home_odds = 1.18, away_odds = 20.06, draw_odds = 8.01 WHERE id = 11707
UPDATE matches SET home_odds = 2.32, away_odds = 3.67, draw_odds = 3.16 WHERE id = 11708
UPDATE matches SET home_odds = 1.32, away_odds = 9.97, draw_odds = 6.27 WHERE id = 11709
UPDATE matches SET home_odds = 1.36, away_odds = 10.1, draw_odds = 5.35 WHERE id = 11705
UPDATE matches SET home_odds = 2.15, away_odds = 4.1, draw_odds = 3.22 WHERE id = 11710
UPDATE matches SET home_odds = 1.7, away_odds = 5.81, draw_odds = 3.85 WHERE id = 11711
UPDATE matches SET home_odds = 12.96, away_odds = 1.26, draw_odds = 6.73 WHERE id = 11704
UPDATE matches SET home_odds = 5.39, away_odds = 1.7, draw_odds = 4.06 WHERE id = 11703
UPDATE matches SET home_odds = 2.46, away_odds = 3.44, draw_odds = 3.11 WHERE id = 11702
UPDATE matches SET home_odds = 7.31, away_odds = 1.52, draw_odds = 4.45 WHERE id = 11699
UPDATE matches SET home_odds = 5.49, away_odds = 1.79, draw_odds = 3.59 WHERE id = 11698
UPDATE matches SET hom

...

UPDATE matches SET home_odds = 16.97, away_odds = 1.17, draw_odds = 7.96 WHERE id = 710747
UPDATE matches SET over_odds = 1.49, under_odds = 2.65 WHERE id = 710747
UPDATE matches SET handicap_size = 2.0, handicap_odds = 1.945 WHERE id = 710747
UPDATE matches SET home_odds = 1.38, away_odds = 8.67, draw_odds = 4.97 WHERE id = 710753
UPDATE matches SET over_odds = 1.7, under_odds = 2.21 WHERE id = 710753
UPDATE matches SET handicap_size = 1.25, handicap_odds = 1.9449999999999998 WHERE id = 710753
UPDATE matches SET home_odds = 6.01, away_odds = 1.53, draw_odds = 4.57 WHERE id = 710756
UPDATE matches SET over_odds = 1.62, under_odds = 2.38 WHERE id = 710756
UPDATE matches SET handicap_size = 1.0, handicap_odds = 1.9449999999999998 WHERE id = 710756
UPDATE matches SET home_odds = 5.42, away_odds = 1.62, draw_odds = 4.25 WHERE id = 710765
UPDATE matches SET over_odds = 1.79, under_odds = 2.08 WHERE id = 710765
UPDATE matches SET handicap_size = 0.75, handicap_odds = 1.95 WHERE id = 710765
U

UPDATE matches SET home_odds = 1.78, away_odds = 4.92, draw_odds = 3.89 WHERE id = 710853
UPDATE matches SET over_odds = 1.76, under_odds = 2.16 WHERE id = 710853
UPDATE matches SET handicap_size = 0.75, handicap_odds = 1.96 WHERE id = 710853
UPDATE matches SET home_odds = 1.13, away_odds = 23.62, draw_odds = 9.86 WHERE id = 710861
UPDATE matches SET over_odds = 1.36, under_odds = 3.29 WHERE id = 710861
UPDATE matches SET handicap_size = 2.25, handicap_odds = 1.9549999999999998 WHERE id = 710861
UPDATE matches SET home_odds = 1.53, away_odds = 7.94, draw_odds = 4.08 WHERE id = 710856
UPDATE matches SET over_odds = 2.09, under_odds = 1.82 WHERE id = 710856
UPDATE matches SET handicap_size = 1.0, handicap_odds = 1.96 WHERE id = 710856
UPDATE matches SET home_odds = 15.0, away_odds = 1.23, draw_odds = 6.63 WHERE id = 710857
UPDATE matches SET over_odds = 1.62, under_odds = 2.41 WHERE id = 710857
UPDATE matches SET handicap_size = 1.75, handicap_odds = 1.955 WHERE id = 710857
UPDATE matche

UPDATE matches SET home_odds = 7.54, away_odds = 1.44, draw_odds = 5.0 WHERE id = 710876
UPDATE matches SET over_odds = 1.64, under_odds = 2.38 WHERE id = 710876
UPDATE matches SET handicap_size = 1.25, handicap_odds = 1.96 WHERE id = 710876
UPDATE matches SET home_odds = 5.15, away_odds = 1.68, draw_odds = 4.25 WHERE id = 710878
UPDATE matches SET over_odds = 1.65, under_odds = 2.36 WHERE id = 710878
UPDATE matches SET handicap_size = 0.75, handicap_odds = 1.965 WHERE id = 710878
UPDATE matches SET home_odds = 1.45, away_odds = 7.2, draw_odds = 5.12 WHERE id = 710762
UPDATE matches SET over_odds = 1.61, under_odds = 2.46 WHERE id = 710762
UPDATE matches SET handicap_size = 1.25, handicap_odds = 1.96 WHERE id = 710762
UPDATE matches SET home_odds = 4.43, away_odds = 1.88, draw_odds = 3.73 WHERE id = 710854
UPDATE matches SET over_odds = 1.93, under_odds = 1.98 WHERE id = 710854
UPDATE matches SET handicap_size = 0.5, handicap_odds = 1.96 WHERE id = 710854
UPDATE matches SET home_odds =

UPDATE matches SET handicap_size = 0.75, handicap_odds = 1.96 WHERE id = 868055
UPDATE matches SET home_odds = 1.48, away_odds = 7.26, draw_odds = 4.67 WHERE id = 868054
UPDATE matches SET over_odds = 1.78, under_odds = 2.15 WHERE id = 868054
UPDATE matches SET handicap_size = 1.25, handicap_odds = 1.975 WHERE id = 868054
UPDATE matches SET home_odds = 4.78, away_odds = 1.85, draw_odds = 3.65 WHERE id = 868046
UPDATE matches SET over_odds = 2.1, under_odds = 1.81 WHERE id = 868046
UPDATE matches SET handicap_size = 0.75, handicap_odds = 1.96 WHERE id = 868046
UPDATE matches SET home_odds = 6.23, away_odds = 1.56, draw_odds = 4.49 WHERE id = 868049
UPDATE matches SET over_odds = 1.65, under_odds = 2.37 WHERE id = 868049
UPDATE matches SET handicap_size = 1.0, handicap_odds = 1.9649999999999999 WHERE id = 868049
UPDATE matches SET home_odds = 2.04, away_odds = 3.8, draw_odds = 3.71 WHERE id = 868052
UPDATE matches SET over_odds = 1.76, under_odds = 2.17 WHERE id = 868052
UPDATE matches S

UPDATE matches SET home_odds = 1.29, away_odds = 10.83, draw_odds = 6.02 WHERE id = 868134
UPDATE matches SET over_odds = 1.56, under_odds = 2.55 WHERE id = 868134
UPDATE matches SET handicap_size = 1.75, handicap_odds = 1.955 WHERE id = 868134
UPDATE matches SET home_odds = 2.1, away_odds = 3.82, draw_odds = 3.52 WHERE id = 868135
UPDATE matches SET over_odds = 2.06, under_odds = 1.85 WHERE id = 868135
UPDATE matches SET handicap_size = 0.5, handicap_odds = 1.9700000000000002 WHERE id = 868135
UPDATE matches SET home_odds = 2.65, away_odds = 2.73, draw_odds = 3.59 WHERE id = 868132
UPDATE matches SET over_odds = 1.82, under_odds = 2.09 WHERE id = 868132
UPDATE matches SET handicap_size = 0.0, handicap_odds = 1.96 WHERE id = 868132
UPDATE matches SET home_odds = 1.99, away_odds = 3.93, draw_odds = 3.76 WHERE id = 868127
UPDATE matches SET over_odds = 2.08, under_odds = 1.84 WHERE id = 868127
UPDATE matches SET handicap_size = 0.5, handicap_odds = 1.96 WHERE id = 868127
UPDATE matches S

In [149]:
execute_multiple_queries(update_queries)